# Kaggle
## 개요
[Kaggle](https://www.kaggle.com/)은 내가 만든 머신러닝 모델을 실제 데이터에 적용하여 테스트 할 수 있는 사이트이다. 주어진 문제에 대해 얼마나 좋은 성능을 내는 모델을 개발해 정답에 가까운 예측을 하는지 여부가 Leaderboard에 기록되기도 한다. 많은 Kaggle 경험 자체가 포트폴리오가 될 수 있는 매우 강력한 서비스이다.

사이트를 들어가면 여러 사람들이 공유한 학습 방법과 데이터 전처리 방법이 있고 커뮤니티가 잘 활성화 되어있다. 이를 통해 많은 것을 배울 수도 있다.

실전에서는 모델을 잘 다루는 것 만큼 중요한 것이 데이터를 정제하는 **데이터 전처리** 작업이다. 실습을 통해 알아보자.

## 실습
입문자들이 하기 가장 좋은 Titanic 문제를 이용하여 Kaggle 사용법을 익혀보자. [Titanic](https://www.kaggle.com/c/titanic/data?select=gender_submission.csv)

먼저 링크에 접속하여 문제를 읽고 Download All 버튼을 눌러 데이터를 다운받는다. 다운 받은 후 데이터에 대한 설명을 읽어보자. 배에 올라탄 사람들에 대한 여러 정보가 담겨있는 데이터임을 알 수 있다.

In [142]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [212]:
tr = pd.read_csv("./data/train.csv")
tst = pd.read_csv("./data/test.csv")

tr.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [213]:
tst.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


데이터에 있는 사람들의 정보를 이용해 그 사람의 구조 여부를 예측하는 것이 이번 문제이다.

우리는 오로지 수치 데이터만 필요하기 때문에 Name 과 Ticket 항목은 `drop` 메소드를 통해 제외시키자. PassengerId와 선실 번호를 의미하는 Cabin도 유의미한 항목으로 보여지지 않기 때문에 제외시키자.

In [214]:
tr = tr.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1)
tst = tst.drop(["PassengerId", "Name", "Ticket", "Cabin"], axis=1)

tr.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


이 후 비어있는 데이터가 있는지 `isna` 메소드로 확인하자. 데이터가 비어있으면 모델 적용 할 때 오류가 날 것이다.

In [215]:
tr.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [216]:
tst.isna().sum()

Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Embarked     0
dtype: int64

비어있는 데이터 처리 방법은 여러가지가 있다. 가장 쉬운 방법은 하나라도 비어있는 데이터가 있으면 그 행을 지워버리는 것이다. 그러나 실전에서는 데이터 하나가 아까운 경우가 많다. 따라서 이번에는 적절한 값으로 빈 데이터를 채워넣는 방법을 이용할 것이다. 가장 간단한 방법으로는 해당 요소에 대한 평균값으로 채워 넣는 것이다. 이 외에도 거리함수를 이용해 가장 가까운 데이터의 요소 값으로 채워넣는 등, 여러가지 방법이 있겠지만 일단 가장 간단한 방법으로 진행 해보자. 결측값을 채우는 것은 `fiilna` 메소드로 할 수 있다.

In [217]:
meanAge_tr = tr.mean()["Age"]
meanAge_tst = tst.mean()["Age"]
fare_tst = tst.mean()["Fare"]

meanAge_tr, meanAge_tst, fare_tst

(29.69911764705882, 30.272590361445783, 35.627188489208635)

In [218]:
tr["Age"] = tr["Age"].fillna(meanAge_tr)
tst["Age"] = tst["Age"].fillna(meanAge_tst)
tst["Fare"] = tst["Fare"].fillna(fare_tst)

Embarked 는 범주형 데이터이기 때문에 평균값 대신 최빈값으로 채울 수 있다.

In [219]:
embarked_tr = tr.mode()["Embarked"]

embarked_tr

0    S
Name: Embarked, dtype: object

In [220]:
tr["Embarked"] = tr["Embarked"].fillna("S")

In [221]:
tr.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [222]:
tst.isna().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [223]:
tr.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


범주형 데이터인 Pclass, Sex, Embarked 를 수치로 바꾸자. Sex와 Embarked 는 문자열이기 때문에 수치로 바꿀 필요가 있지만, Pclass 의 값은 1, 2, 3이기 때문에 1만 빼주면 된다. 데이터를 특정 값으로 변환하는 것은 `apply` 메소드를 통해 실행 할 수 있다.

In [224]:
map_sex = { "male": 0, "female": 1 }
map_embarked = { "C": 0, "Q": 1, "S": 2 }

for df in [tr, tst]:
    df["Pclass"] = df["Pclass"].apply(lambda x: x - 1)
    df["Sex"] = df["Sex"].apply(lambda x: map_sex[x])
    df["Embarked"] = df["Embarked"].apply(lambda x: map_embarked[x])
    
tr.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,2,0,22.0,1,0,7.2500,2
1,1,0,1,38.0,1,0,71.2833,0
2,1,2,1,26.0,0,0,7.9250,2
3,1,0,1,35.0,1,0,53.1000,2
4,0,2,0,35.0,0,0,8.0500,2


In [225]:
tr.dtypes

Survived      int64
Pclass        int64
Sex           int64
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked      int64
dtype: object

전부 수치 데이터로 바뀐 것을 알 수 있다. 이제 범주형 데이터를 one-hot encoding 해서 추가하자.

In [226]:
cols = ["Pclass", "Sex", "Embarked"]
counts = [3, 2, 3] # 범주 개수

tr_ = pd.DataFrame(tr)
tst_ = pd.DataFrame(tst)

for c, count in zip(cols, counts):
    for df in [tr_, tst_]:
        n = df.shape[0]
        arr = np.zeros((n, count))
        arr[np.arange(n), df[c]] = 1
        for i in range(count):
            df[c + "_" + str(i)] = arr[:, i]
            
tr_.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Pclass_0,Pclass_1,Pclass_2,Sex_0,Sex_1,Embarked_0,Embarked_1,Embarked_2
0,0,2,0,22.0,1,0,7.2500,2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
1,1,0,1,38.0,1,0,71.2833,0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,1,2,1,26.0,0,0,7.9250,2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,1,0,1,35.0,1,0,53.1000,2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,0,2,0,35.0,0,0,8.0500,2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0


범주형 데이터에 대한 정보는 one-hot encoding 이 된 열에 있기 때문에 원래 데이터는 삭제를 해주어도 된다.

Age, Fare는 다른 데이터와 단위가 다르기 때문에 표준화 해 주자.

In [227]:
for df in [tr_, tst_]:
    for c in ["Age", "Fare"]:
        df[c] = (df[c] - df[c].mean()) / df[c].std()
    df = df.drop(["Pclass", "Sex", "Embarked"], axis=1)
    
tr_.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Pclass_0,Pclass_1,Pclass_2,Sex_0,Sex_1,Embarked_0,Embarked_1,Embarked_2
0,0,2,0,-0.592148,1,0,-0.502163,2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
1,1,0,1,0.638430,1,0,0.786404,0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
2,1,2,1,-0.284503,0,0,-0.488580,2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,1,0,1,0.407697,1,0,0.420494,2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,0,2,0,0.407697,0,0,-0.486064,2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0


다중공선성 확인을 위해 상관계수를 구해보자. 만약 상관관계가 높은 열이 있다면 삭제를 고려 할 수 있다. 상관계수의 절댓값이 0.5보다 높은 경우를 출력 해보자.

In [228]:
for n, i in enumerate(tr_.columns):
    for j in tr_.columns[n + 1:]:
        if i == j or  i == "Survived" or j == "Survived":
            continue
        if abs(tr_.corr()[i][j]) > 0.5:
            print(i, j, tr_.corr()[i][j])

Pclass Fare -0.5494996199439073
Pclass Pclass_0 -0.8859238618875533
Pclass Pclass_2 0.9166727522685237
Sex Sex_0 -0.9999999999999999
Sex Sex_1 1.0
Fare Pclass_0 0.591710718883511
Embarked Embarked_0 -0.9362730393120979
Embarked Embarked_2 0.951473826059107
Pclass_0 Pclass_2 -0.6267375846396959
Pclass_1 Pclass_2 -0.565210153553826
Sex_0 Sex_1 -1.0
Embarked_0 Embarked_2 -0.7827421287187312


Fare 가 Pclass_0 과 높은 상관관계를 가지고 있음을 볼 수 있다. 따라서 Fare 를 삭제 해주자.
- 사실 다중공선성 문제에 대처 할 때에는, 위와 같이 무작정 삭제하는 방법을 좋지 않고, 실제 데이터를 시각화 하여 판단하거나, 도메인 지식을 근거로 삭제하는 등 더욱 심도 있는 작업이 필요하다.

In [229]:
tr_ = tr_.drop(["Fare"], axis=1)
tst_ = tst_.drop(["Fare"], axis=1)

데이터 전처리 작업을 마치고 모델을 선택 해 보자. 먼저 대표적인 분류 모델인 로지스틱 회귀모델을 이용 해보자.

In [300]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [301]:
X = np.array(tr_.drop(["Survived"], axis=1))
y = np.array(tr_["Survived"])

X.shape

(891, 14)

In [302]:
X_tr, X_tst, y_tr, y_tst = train_test_split(X, y, test_size=0.3)

X_tr.shape, X_tst.shape

((623, 14), (268, 14))

In [303]:
model = LogisticRegression()
model.fit(X_tr, y_tr)

y_hat = model.predict(X_tst)

np.sum(y_tst==y_hat) / y_hat.shape[0]

0.8283582089552238

다음으로 knn 모델을 써보자.

In [304]:
from sklearn.neighbors import KNeighborsClassifier

In [305]:
for k in range(1, 25, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_tr, y_tr)
    y_hat = knn.predict(X_tst)
    
    print(k, np.sum(y_tst==y_hat) / y_tst.shape[0])

1 0.7574626865671642
3 0.7723880597014925
5 0.7798507462686567
7 0.8134328358208955
9 0.8022388059701493
11 0.8059701492537313
13 0.8022388059701493
15 0.8059701492537313
17 0.8022388059701493
19 0.8022388059701493
21 0.7985074626865671
23 0.7985074626865671


K=11 일 때의 모델이 가장 좋은 성능을 보인다.

두 모델을 합쳐서 테스트 해보자. 즉, 두 모델이 출력하는 확률의 평균을 이용 해 보자.

In [306]:
knn = KNeighborsClassifier(n_neighbors=11)
knn.fit(X_tr, y_tr)

log_prob = model.predict_proba(X_tst)
knn_prob = knn.predict_proba(X_tst)

y_prob = (log_prob + knn_prob) / 2
y_hat = np.argmax(y_prob, axis=1)

np.sum(y_tst==y_hat) / y_tst.shape[0]

0.832089552238806

정답 파일을 만들어 제출한다.

In [310]:
tst_original = pd.read_csv("./data/test.csv")
ans = pd.DataFrame()

log_prob = model.predict_proba(np.array(tst_))
knn_prob = knn.predict_proba(np.array(tst_))
y_prob = (log_prob + knn_prob) / 2
y_hat = np.argmax(y_prob, axis=1)

ans["PassengerID"] = tst_original["PassengerId"]
ans["Survived"] = y_hat

ans.head()

,PassengerID,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [311]:
ans.to_csv("./data/ans.csv", index=False)

위 모델을 0.77990 점을 받았다. 더 높은 점수를 받을 수 있는 방법을 생각해보자.